In [4]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Define the file path
file_path = './1hz3_T310.run.25000000.energy.xvg'

# Read the file and ignore header lines that start with '@' or '#'
data = pd.read_csv(file_path, comment='@', delim_whitespace=True, header=None, skiprows=lambda x: x<24)

# Add column names based on the header in the file
data.columns = ['Time', 'Potential', 'Kinetic', 'Total_Energy', 'Temperature', 'Pressure', 'Volume']

# Convert time from picoseconds to nanoseconds
data['Time'] = data['Time'] / 1000  # converting ps to ns

# Define the PDF file to save plots
pdf_file = 'energy_plots.pdf'

# Use PdfPages to save multiple plots to a single PDF file
with PdfPages(pdf_file) as pdf:
    
    # Part b: Plot each quantity against time (6 separate plots)
    columns_to_plot = ['Potential', 'Kinetic', 'Total_Energy', 'Temperature', 'Pressure', 'Volume']

    # Loop through columns and create plots
    for col in columns_to_plot:
        plt.figure()
        plt.plot(data['Time'], data[col])
        plt.title(f'{col} vs Time')
        plt.xlabel('Time (ns)')
        plt.ylabel(col)
        plt.grid(True)
        pdf.savefig()  # Save the current figure to the PDF file
        plt.close()    # Close the figure to free memory
    
    # Part c: Plot Kinetic Energy and Temperature on the same plot with dual axes
    fig, ax1 = plt.subplots()

    # Plot Temperature on the left axis
    ax1.set_xlabel('Time (ns)')
    ax1.set_ylabel('Temperature (K)', color='tab:blue')
    ax1.plot(data['Time'], data['Temperature'], color='tab:blue', label='Temperature')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    # Create a second y-axis for Kinetic Energy
    ax2 = ax1.twinx()
    ax2.set_ylabel('Kinetic Energy (kJ/mol)', color='tab:red')
    ax2.plot(data['Time'], data['Kinetic'], color='tab:red', label='Kinetic Energy')
    ax2.tick_params(axis='y', labelcolor='tab:red')

    fig.tight_layout()  # Ensure no overlap of labels
    plt.title('Temperature and Kinetic Energy vs Time')
    plt.grid(True)
    pdf.savefig()  # Save the dual-axis plot to the PDF file
    plt.close()    # Close the figure


In [1]:
# Import necessary libraries
import mdtraj as md
import matplotlib.pyplot as plt
from fpdf import FPDF

# Define the PDF file to save the report
pdf_file = 'ubiquitin_analysis.pdf'

# Define the PDB file path for Ubiquitin structure (You need to set the correct path to the PDB file)
pdb_file = './1UBQ_processed.pdb'

# Part a: Read in the ubiquitin structure using mdtraj
# Load the PDB file using mdtraj
traj = md.load(pdb_file)

# Part b: Print the total number of hydrogen bonds
# Compute the hydrogen bonds using mdtraj's `wernet_nilsson` method
h_bonds = md.baker_hubbard(traj, periodic=False)

# Get the number of hydrogen bonds
num_h_bonds = len(h_bonds)
print(f'Total number of hydrogen bonds: {num_h_bonds}')

# Part c: Compute the number of helical amino acids
# Compute the secondary structure using the DSSP algorithm
dssp = md.compute_dssp(traj)

# Count the number of residues in alpha-helix (denoted by 'H' in DSSP)
helical_residues = (dssp == 'H').sum()
print(f'Total number of helical residues: {helical_residues}')

# Output results to a PDF file using fpdf
pdf = FPDF()
pdf.add_page()

# Add title
pdf.set_font('Arial', 'B', 16)
pdf.cell(200, 10, 'Ubiquitin Structure Analysis Report', ln=True, align='C')

# Add the results to the PDF
pdf.set_font('Arial', '', 12)
pdf.ln(10)
pdf.cell(200, 10, f'Total number of hydrogen bonds: {num_h_bonds}', ln=True)
pdf.cell(200, 10, f'Total number of helical residues: {helical_residues}', ln=True)

# Save the PDF
pdf.output(pdf_file)
print(f'Report saved as {pdf_file}')


Total number of hydrogen bonds: 57
Total number of helical residues: 18
Report saved as ubiquitin_analysis.pdf


In [6]:
# Import necessary libraries
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Define the path to the trajectory and topology files
trajectory_file = './1hz3_T310.stepid25000000.every100ps.nowater.xtc'
topology_file = './1hz3_T310.start.nowater.gro'

# Part a: Load the trajectory using mdtraj
traj = md.load(trajectory_file, top=topology_file)

# Part b: Compute end-to-end distance and radius of gyration at each time
# Assuming that the first and last residue of the chain represent the end-to-end distance
# (adjust the residue indices based on your system)
end_to_end_distance = md.compute_distances(traj, [[0, traj.n_atoms - 1]])[:, 0]

# Compute the radius of gyration
radius_of_gyration = md.compute_rg(traj)

# Convert time to nanoseconds (assuming the trajectory time step is 100 ps)
time = np.arange(0, traj.n_frames * 100, 100) / 1000  # Convert ps to ns

# Open a PDF file to save the figures
pdf_filename = 'TASK6.pdf'
with PdfPages(pdf_filename) as pdf:

    # Part c: Plot end-to-end distance and radius of gyration vs time on the same plot
    plt.figure()
    plt.plot(time, end_to_end_distance, label='End-to-End Distance (nm)')
    plt.plot(time, radius_of_gyration, label='Radius of Gyration (nm)')
    plt.xlabel('Time (ns)')
    plt.ylabel('Distance (nm)')
    plt.title('End-to-End Distance and Radius of Gyration vs Time')
    plt.legend()
    plt.grid(True)
    pdf.savefig()  # Save the current figure to the PDF
    plt.close()

    # Part d: Plot normalized histograms of end-to-end distance and radius of gyration

    # Plot histogram of end-to-end distance
    plt.figure()
    plt.hist(end_to_end_distance, bins=30, density=True, alpha=0.7, color='b', label='End-to-End Distance')
    plt.xlabel('End-to-End Distance (nm)')
    plt.ylabel('Probability Density')
    plt.title('Normalized Histogram of End-to-End Distance')
    plt.grid(True)
    plt.legend()
    pdf.savefig()  # Save the current figure to the PDF
    plt.close()

    # Plot histogram of radius of gyration
    plt.figure()
    plt.hist(radius_of_gyration, bins=30, density=True, alpha=0.7, color='r', label='Radius of Gyration')
    plt.xlabel('Radius of Gyration (nm)')
    plt.ylabel('Probability Density')
    plt.title('Normalized Histogram of Radius of Gyration')
    plt.grid(True)
    plt.legend()
    pdf.savefig()  # Save the current figure to the PDF
    plt.close()

print(f"PDF report saved as {pdf_filename}")


PDF report saved as TASK6.pdf


In [9]:
import PyPDF2

# 定义要合并的PDF文件路径
pdfs = ['TASK2.pdf', 'TASK4.pdf', 'TASK5.pdf', 'TASK6.pdf']

# 输出的PDF文件名
output = 'ASSIGNMENT.pdf'

# 创建一个 PdfMerger 对象
merger = PyPDF2.PdfMerger()

# 将所有的 PDF 文件合并到一起
for pdf in pdfs:
    with open(pdf, 'rb') as f:
        merger.append(f)

# 将合并后的文件保存
with open(output, 'wb') as f_out:
    merger.write(f_out)

print(f"所有PDF文件已成功合并为 {output}")


所有PDF文件已成功合并为 ASSIGNMENT.pdf
